## Main Idea

A main challenge in the training of deep neural networks is the dynamic nature of each layer's inputs, which fluctuate as parameters evolve during the training process. This phenomenon, termed "Internal Covariate Shift" by Ioffe and Szegedy (2015), presents a significant challenge to the stability and efficiency of the training process. To address this issue, the authors introduce a novel mechanism known as Batch Normalization, which mitigates the effects of internal covariate shift by standardizing the inputs to each layer. We will explain what standardization means in this context in the later sections.

The central premise of the paper is the proposition of this mechanism that not only accelerates the training process but also offers ancillary benefits such as regularization. This innovative concept has proven instrumental in facilitating the training of deep neural networks, enabling the authors to achieve state-of-the-art results on benchmark datasets. The significance of this contribution cannot be overstated, as it has greatly advanced the landscape of deep learning methodologies.

## Summary

### Internal Covariate Shift

Internal Covariate Shift is a phenomenon that occurs during the training of deep neural networks, where the distribution of each layer's inputs changes as the parameters of the preceding layers evolve. For instance, consider two layers $l_{1}$ and $l_{2}$ in a network. If the activations of $l_{1}$ initially follow a normal distribution $a_{1} \sim N(\mu_{1}, \sigma_{1}^{2})$, internal covariate shift occurs when, after some training steps, the activations shift to a different distribution, say $a_{1} \sim N(\mu_{2}, \sigma_{2}^{2})$. This shift in distribution, or instability, impedes the training process as subsequent layers must continually adapt to these changes.

### Normalization

To mitigate the effects of internal covariate shift, Ioffe and Szegedy (2015) propose that the outputs (and hence the inputs) of each layer should be linearly transformed to have zero mean and unit variance. This normalization stabilizes the distribution of the inputs to each layer, reducing the instability during training. However, whitening each layer's inputs can be computationally expensive and, crucially, not everywhere differentiable. To address these issues, the authors propose two simplifications:

1. Normalize each feature independently, making it have zero mean and unit variance. This is a departure from joint whitening of the features in layer inputs and outputs.

2. Compute the estimated means and variances over the mini-batch rather than the full training set (detached). This approach involves the normalization as a part of the optimization, allowing us to compute the gradients with respect to the parameters of the Batch Normalization transform.

These simplifications allow for efficient and differentiable normalization, which is integral to the Batch Normalization technique.

Note that, at training time, the statistics are computed over the mini-batch. At inference time, the typical practice is to compute the exponential moving averages of the means and variances for the mini-batches and use those at inference time. According to the authors, this also has a regularization effect on the model.

### Learnable Parameters

The authors note that whitening process can impact the representations of the layers. For example, if the inputs to a sigmoid function are normalized, the majority of the values will fall within the range of $[-1, 1]$. Consequently, each normalized feature is adjusted and shifted by two modifiable parameters— $\gamma$ and $\beta$. In fact, these parameters have the ability to adjust and shift the normalized features in a manner that could potentially revert them back to their initial values, if that proves to be the best solution.

### Batch Normalization Equations

#### Image Data

For image data $X \in \mathbb{R}^{B \times C \times H \times W}$, the dimensions are as follows:

* B is the batch dimension

* C is the number of channels (e.g. RGB for the input images) or the number of feature maps (created by the filters in the intermediate layer outputs)

* H (height) and W (width) are the spatial dimensions

The batch normalization equation is given as follows:

$$
\mathrm{BN}(X)=\gamma \frac{\boldsymbol{X}-\underset{B, H, W}{\mathbb{E}}[X]}{\sqrt{\underset{B, H, W}{\operatorname{Var}}[\boldsymbol{X}]+\epsilon}}+\beta
$$

where

* $X \in \mathbb{R}^{B \times C \times H \times W}$

* $\underset{B, H, W}{\mathbb{E}}[X] \in \mathbb{R}^{C}$ is the vector of expectations (means) of $X$ computed over the batch dimension $B$ and the spatial dimensions height $H$, and $W$ width for each channel or feature map

* $\underset{B, H, W}{\operatorname{Var}} \in \mathbb{R}^{C}$ is the vector of variances of $X$ computed over the batch dimension $B$ and the spatial dimensions height $H$, and $W$ width for each channel or feature map 

* $\gamma \in \mathbb{R}^C$ (scale) and $\beta \in \mathbb{R}^C$ (shift) are learnable parameter vectors of the batch normalization layer

* $\epsilon$ is a small constant used to guard against division by zero

The subtraction and division operations in the batch normalization formula are performed element-wise. This means that all values (scalars in the arrays) in each of the channels in the input $X$ are normalized by subtracting the corresponding channel's mean (a scalar) and dividing by the corresponding channel's standard deviation (a scalar). This is possible because the vectors of means and variances are broadcasted to have the correct shapes during these operations.

#### Tabular Data

When the input data $X \in \mathbb{R}^{B \times C}$ is a batch of training examples where $B$ is the batch size and $C$ is the number of features or predictors:

$$
\mathrm{BN}(X)=\gamma \frac{X-\underset{B}{\mathbb{E}}[X]}{\sqrt{\operatorname{Var}_B[X]+\epsilon}}+\beta
$$

In this case, the batch normalization procedure is similar to the standardization technique often employed in statistical learning modeling context. There is one mean and variance for each of the $C$ features or predictors computed over the mini-batch $B$. These values are stored in vectors $\underset{B}{\mathbb{E}}[X] \in \mathbb{R}^{C}$ and $\underset{B}{\operatorname{Var}} \in \mathbb{R}^{C}$ and are broadcasted during the computations such that each column in $X$ is subtracting and dividing by the same mean and variance. Finally, each normalized feature or predictor is scaled and shifted by the corresponding $\gamma_{i}$ and $\beta_{j}$ where $i, j \in [1, 2, ..., C]$.

### PyTorch Implementation

In [1]:
import torch
from labml_helpers.module import Module
from torch import nn

In [106]:
class BatchNorm(Module):
    def __init__(
        self,
        channels: int,
        *,
        eps: float = 1e-5,
        momentum: float = 0.1,
        affine: bool = True,
        track_running_stats: bool = True,
    ):
        """
        Constructor for the BatchNorm class.

        Parameters
        ----------
        channels : int
            Number of feature maps in the input.
        eps : float, optional
            A small constant for numerical stability, by default 1e-5.
        momentum : float, optional
            The momentum in taking the exponential moving average, by default 0.1.
        affine : bool, optional
            Whether to scale and shift the normalized activations, by default True.
        track_running_stats : bool, optional
            whether to compute the moving averages of mean and variance, by default True.
        """
        super().__init__()

        self.channels = channels
        self.eps = eps
        self.momentum = momentum
        self.affine = affine
        self.track_running_stats = track_running_stats

        if self.affine:
            # Initialize the scale (gammas) and shift (betas) parameters (vectors with torch.Size([self.channels]) as shapes)
            self.scale = nn.Parameter(torch.ones(channels))
            # We could alternatively used self.register_parameter(name, param) to register the parameters, but we have to pass the name as a string
            self.shift = nn.Parameter(torch.zeros(channels))

        if self.track_running_stats:
            # Zero means, and these should be restored in 'state_dict' but not trained by the optimizer
            self.register_buffer("exp_mean", torch.zeros(channels))
            # Unit variances, and these buffers would not be returned by model.parameters()
            self.register_buffer("exp_var", torch.ones(channels))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the BatchNorm layer, which is called by __call__ when as use BatchNorm as a function.

        Parameters
        ----------
        x : torch.Tensor
            The input tensor.

        Returns
        -------
        torch.Tensor
            The normalized tensor.
        """
        # For images, the shape is `[batch_size, channels, height, width]`
        x_original_shape = x.shape
        # Expect the first dimension to be the batch dimension
        batch_size = x_original_shape[0]

        # Expect the second dimension to be the number of features or channels
        assert self.channels == x.shape[1]

        # Reshape to `[batch_size, channels, n]` where `n` is the product of the remaining dimensions (height and width) since -1 is inferred from the remaining (unspecified) dimensions
        # The torch.Tensor.view(*shape) method returns a new tensor with the same data as the self tensor but of a different shape
        x = x.view(batch_size, self.channels, -1)

        # The self.training attribute is a bool value that is set to True when the model is in training mode, and False when the model is in evaluation mode
        # This if block will be executed when we are either in training mode OR we are not computing moving averages
        if self.training or not self.track_running_stats:
            # Calculate the mean across first (batch) and last dimension (n), i.e. the means for each feature E[x^k] (resulting in a vector with size `self.channels`)
            # Note that the missing dimension is inferred from the remaining dimensions (1 or the channel dimension)
            mean = x.mean(dim=[0, 2])
            # Calculate the mean of squared x across first and last dimension, i.e. the squared means for each feature E[(x^k)^2] (a vector with size `self.channels`)
            # Note this is computed by first squaring x element-wise and then taking the mean across the first and last dimension
            mean_x2 = (x**2).mean(dim=[0, 2])
            # Variance for each feature Var[x^k] = E[(x^k)^2] - (E[x^k])^2 (a vector with size `self.channels`)
            var = mean_x2 - mean**2

            # This if block will be run when we are in training mode AND we are computing moving averages
            if self.training and self.track_running_stats:
                # The multiples (1 - self.momentum) and self.momentum are scalars, and the mean and var are vectors with size `self.channels`
                # Note var and mean are the current batch statistics and self.exp_var and self.exp_mean are the moving averages from previous batches
                self.exp_mean = (
                    1 - self.momentum
                ) * self.exp_mean + self.momentum * mean
                self.exp_var = (1 - self.momentum) * self.exp_var + self.momentum * var

        # Else block will be executed when we are not in training mode (eval mode) AND we are computing moving averages
        else:
            # We do not update the mean and variance when in eval mode, so the variables 'mean' and 'variance' will point to moving averages from previous batches
            mean = self.exp_mean
            var = self.exp_var

        # Normalize x_hat^k = (x^k - E[x^k]) / sqrt(Var[x^k] + eps), which is an array with the same shape as `x`, i.e., torch.Size([batch_size, self.channels, height * width])
        x_norm = (x - mean.view(1, -1, 1)) / torch.sqrt(var + self.eps).view(1, -1, 1)
        # Scale and shift y^k = gamma^k * x_hat^k + beta^k (a vector with size `self.channels`)
        if self.affine:
            x_norm = self.scale.view(1, -1, 1) * x_norm + self.shift.view(1, -1, 1)

        # Reshape to original and return
        return x_norm.view(x_original_shape)

For image data:

* When we reshape with `-1`, we actually flatten the array. For examples if `x` has shape `[2, 3, 4, 5]`, `x.view(2, 3, -1)` flattens the 2D `4 x 5` matrix into a 1D vector with `20` elements. The resulting array has shape `[2, 3, 20]`. When we reshape this array back to the original shape of `[2, 3, 4, 5]` (e.g. `x_norm.view(x_original_shape)`), the the 1D vector with `20` elements will be reshaped to a matrix with shape `[4, 5]`.

* The equation for normalization is:
  
  $$\hat{x}^{(k)}=\frac{x^{(k)}-\mathbb{E}\left[x^{(k)}\right]}{\sqrt{\operatorname{Var}\left[x^{(k)}\right]+\epsilon}}$$

  Note that mean and variance vectors are reshaped (broadcasted via `view(1, -1, 1)`) to have shape `torch.Size([1, self.channel, 1])` in order to be compatible with `x`. Recall that `x` was reshaped to have shape `torch.Size([batch_size, self.channels, height * width])` earlier. The channel dimension matched and so the operations are performed element-wise across the first and the last dimension. The output is an array $\hat{x}^{(k)}$ that has the same shape as the input `x`.

* The scale and shift equation is:

  $$y^{(k)}=\gamma^{(k)} \hat{x}^{(k)}+\beta^{(k)}$$

  Note that the scale and shift vectors with sizes `torch.Size[self.channels]` are also reshaped to `torch.Size([1, self.channel, 1])` to be compatible with $x$.

Test:

In [111]:
def test():
    """
    Simple test
    """
    # Create a batch of 2 images with 3 channels, height 2 and width 4
    x = torch.rand([2, 3, 2, 4])
    print(f"X: {x}")
    bn = BatchNorm(channels=3)

    x = bn(x)

    print(f"x.shape: {x.shape}")
    print(f"bn.exp_mean: {bn.exp_mean}")
    print(f"bn.exp_var: {bn.exp_var}")


test()

X: tensor([[[[0.2936, 0.1668, 0.8981, 0.6267],
          [0.6978, 0.9425, 0.9763, 0.9682]],

         [[0.9170, 0.1710, 0.4670, 0.1775],
          [0.7568, 0.4677, 0.2522, 0.0197]],

         [[0.3884, 0.2906, 0.6723, 0.2479],
          [0.2997, 0.3152, 0.0100, 0.3083]]],


        [[[0.7291, 0.7572, 0.6607, 0.1274],
          [0.3151, 0.1659, 0.0620, 0.2196]],

         [[0.3425, 0.1818, 0.2700, 0.4802],
          [0.3300, 0.1906, 0.3855, 0.3049]],

         [[0.2300, 0.5868, 0.1815, 0.9072],
          [0.6629, 0.8549, 0.6594, 0.9906]]]])
x.shape: torch.Size([2, 3, 2, 4])
bn.exp_mean: tensor([0.0538, 0.0357, 0.0475])
bn.exp_var: tensor([0.9105, 0.9048, 0.9079])


## Approach & Contributions

The authors, Ioffe and Szegedy, argue that Batch Normalization (BN) significantly enhances the efficiency and performance of deep neural networks by mitigating a key challenge in their training processes. Their empirical analysis reveals several facets of this accelerated training:

* BN permits the use of larger learning rates, facilitating larger updates without adverse effects.

* It enables a simpler architecture by eliminating the need for dropout layers and reducing $L_{2}$ regularization. This is possible because BN behaves differently during training and inference modes.

* When used in conjunction with a learning rate scheduler, BN allows for a higher decay rate, potentially leading to faster convergence.

* Models employing BN require less data augmentation preprocessing. The authors rationalize this by explaining that networks with BN train faster, thereby observing each training example fewer times. Consequently, training can focus more on real images.

This work builds upon the existing understanding of the challenges in training deep neural networks, particularly the problem of internal covariate shift, which was known to hinder deep network training. By reducing internal covariate shift, BN simplifies the training of deep networks, potentially leading to more powerful and efficient models. Today, batch normalization is observed in some of the most state-of-the-art convolutional neural network architectures for computer vision tasks.

## Areas for Improvements

While the paper "Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift" by Ioffe and Szegedy has made significant contributions to the field of deep learning, there are a few areas that could be improved or clarified:

* **Ambiguity in Practical Application**: One of the main areas of ambiguity in the paper is the practical application of batch normalization, particularly concerning whether it should be applied before or after the activation function. The paper does not provide clear guidance on this matter, leading to debates among practitioners. Future work could focus on providing empirical evidence to resolve this ambiguity, which would be of great help to practitioners in the field. There is a github [issue](https://github.com/keras-team/keras/issues/1802#issuecomment-187966878) that captures this debate.

* **Theoretical Understanding**: The paper posits that batch normalization operates by "reducing internal covariate shift." However, the exact mechanism by which batch normalization improves the performance of deep networks remains unclear. While there are various hypotheses, none have been definitively proven. Future research could focus on developing a more thorough theoretical understanding of why and how batch normalization works.

* **Experimental Setup**: While the paper provides empirical evidence of the effectiveness of batch normalization, the experimental setup could be expanded to include a wider variety of network architectures and datasets beyond image classification. This would provide a more comprehensive understanding of the contexts in which batch normalization is most effective. Although practitioners have most often observed the effective of this technique in training deep networks. 

## Reference

* Ioffe, Sergey, and Christian Szegedy. 2015. "Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift." arXiv preprint arXiv:1502.03167. https://arxiv.org/abs/1502.03167.

* Jayasiri, Varuna, and Nipun Wijerathne. 2020. "labml.ai Annotated Paper Implementations." Accessed [insert date you accessed the source]. https://nn.labml.ai/.